In [1]:
# Load general utilities
# ----------------------
import pandas as pd
from scipy import stats
import datetime
import math
import numpy as np
import pickle

# Load the data and engineer features

In [2]:
# This is the code you can use to open your pickle file
# Read the data and features from the pickle
final_data, discrete_features, continuous_features, ret_cols = pickle.load( open( "Data/clean_data.pickle", "rb" ) )

In [3]:
# Create a feature for the length of a person's credit history at the
# time the loan is issued
final_data['cr_hist'] = (final_data.issue_d - final_data.earliest_cr_line) / np.timedelta64(1, 'M')
continuous_features.append('cr_hist')

# smaller sample of the data due to time constraints


In [4]:
# this code randomly samples 55% of the rows
# change the frac paramter if you want a different % to sample
# replace = False insures we won't select the same row twice
final_data=final_data.sample(frac=.25, replace=False, ).copy()

# Function to Calculate PValues¶¶

In [5]:
def getPValues (model, X_test, y_test):
    params = np.append(model.intercept_,model.coef_)
    predictions = model.predict(X_test)

    newX = pd.DataFrame({"Constant":np.ones(len(X_test))}).join(pd.DataFrame(X_test.reset_index(drop=True)))

    mse = mean_squared_error(y_test, predictions)

    var_b = mse*(np.linalg.pinv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b
    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX.columns)-1))) for i in ts_b]
    sd_b = np.round(sd_b,3)
    ts_b = np.round(ts_b,3)
    p_values = np.round(p_values,8)
    params = np.round(params,4)

    df = pd.DataFrame()
    df["Coeff"],df["SE"],df["t val"],df["Probs"] = [params,sd_b,ts_b,p_values]
    names = ['Intercept']
    names.extend(list(X_test))
    df.index = names
    return df

# X and y from the features code cells

In [6]:
from sklearn.preprocessing import MinMaxScaler

def minMaxScaleContinuous(continuousList):
    return pd.DataFrame(MinMaxScaler().fit_transform(final_data[continuousList])
                             ,columns=list(final_data[continuousList].columns)
                             ,index = final_data[continuousList].index)

def createDiscreteDummies(discreteList):
    return pd.get_dummies(final_data[discreteList], dummy_na = True, prefix_sep = "::", drop_first = False)

# Define which features to use in the modeling

In [7]:
# define the discrete features you want to use in modeling.
# if you want to use all the discrete features, just set discrete_features_touse = discrete_features
discrete_features_touse =['purpose', 'term', 'verification_status', 'emp_length', 'home_ownership']

# define the continuous features to use in modeling
# if you want to use all the continuous features, just set the continuous_features_touse = continuous_features
continuous_features_touse = ['loan_amnt', 'funded_amnt','installment','annual_inc','dti','revol_bal','delinq_2yrs','open_acc',
 'pub_rec','fico_range_high','fico_range_low','revol_util','cr_hist']

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Create dummies for categorical features and concatenate with continuous features for X or predictor dataframe

# Use this line of code if you do not want to scale the continuous features
#X_continuous = final_data[continuous_features_touse]

# use this line if you want to scale the continuous features using the MinMaxScaler in the function defined above
X_continuous = minMaxScaleContinuous(continuous_features_touse)

# create numeric dummy features for the discrete features to be used in modeling
X_discrete = createDiscreteDummies(discrete_features_touse)

#concatenate the continuous and discrete features into one dataframe
X = pd.concat([X_continuous, X_discrete], axis = 1)

# this is the target variable 
# 'ret_PESS', 'ret_OPT', 'ret_INTa', 'ret_INTb'

# Use this line of code if you do not want to scale the ret_cols
#=final_data["ret_INTa"]

# use this line if you want to scale the ret_cols using the MinMaxScaler in the function defined above
ret_data = minMaxScaleContinuous(ret_cols)
y=ret_data['ret_INTa']

# create a test and train split of the transformed data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=.3)


# Multiple Linear Regression to predict M3(ret_INTa)

In [9]:
from sklearn.linear_model import LinearRegression

mlr_model = LinearRegression(n_jobs=-1).fit(X_train, y_train)

print("mlr.coef_:", mlr_model.coef_)
print("mlr.intercept_:", mlr_model.intercept_)

mlr.coef_: [ 6.64097398e-02  6.57001381e-02 -2.51232682e-01  3.14640281e-01
 -2.32962189e-01  1.93620141e-01 -1.27526009e-02 -2.13605483e-02
  6.07298681e-02 -4.16886036e-01  4.53854713e-01  1.16077699e-02
  7.99493913e-03  3.65607148e+10  3.65607148e+10  3.65607148e+10
  3.65607148e+10  3.65607148e+10  3.65607148e+10  3.65607148e+10
  3.65607148e+10  3.65607148e+10  3.65607148e+10  3.65607148e+10
  3.65607148e+10  3.65607148e+10 -1.90666318e+10  6.09349937e+09
  6.09349937e+09 -4.85159857e+10 -7.61518483e+11 -7.61518483e+11
 -7.61518483e+11  2.59394008e+10 -1.23224121e+10 -1.23224121e+10
 -1.23224121e+10 -1.23224121e+10 -1.23224121e+10 -1.23224121e+10
 -1.23224121e+10 -1.23224121e+10 -1.23224121e+10 -1.23224121e+10
 -1.23224121e+10 -1.23224121e+10  8.96350763e+11  8.96350763e+11
  8.96350763e+11  8.96350763e+11  0.00000000e+00]
mlr.intercept_: -165164082541.41138


In [10]:
print("Training set score: {:.5f}".format(mlr_model.score(X_train, y_train)))
print("Test set score: {:.5f}".format(mlr_model.score(X_test, y_test)))


Training set score: 0.02088
Test set score: 0.02238


In [11]:
import math
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error

predictions = mlr_model.predict(X_test)
score = explained_variance_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = math.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print("score = {:.5f} | MAE = {:.5f} | RMSE = {:.5f} | R2 = {:.5f}".format(score, mae, rmse, r2))

score = 0.02238 | MAE = 0.12389 | RMSE = 0.16964 | R2 = 0.02238


In [12]:
getPValues(mlr_model, X_train, y_train)

C:\Users\coolr\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
C:\Users\coolr\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\coolr\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\coolr\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


,Coeff,SE,t val,Probs
Intercept,-1.651641e+11,0.006,-2.682357e+13,0.000000e+00
loan_amnt,6.640000e-02,0.006,1.161100e+01,0.000000e+00
funded_amnt,6.570000e-02,0.006,1.148700e+01,0.000000e+00
installment,-2.512000e-01,0.015,-1.641800e+01,0.000000e+00
annual_inc,3.146000e-01,0.059,5.339000e+00,9.000000e-08
dti,-2.330000e-01,0.036,-6.530000e+00,0.000000e+00
revol_bal,1.936000e-01,0.039,4.931000e+00,8.200000e-07
delinq_2yrs,-1.280000e-02,0.014,-9.150000e-01,3.600369e-01
open_acc,-2.140000e-02,0.007,-3.018000e+00,2.541370e-03
pub_rec,6.070000e-02,0.036,1.707000e+00,8.781582e-02


# LASSO Regression GridsearchCV

In [13]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

''' These are just example parameter settings. You can change these parameters or add others.
    The grid search uses a scoring method of R2. You can change that to another scoring method.
'''

parameters = {'alpha' : [0.000000001, 0.00000001, 0.0000001]
             }

print("Parameter grid:\n{}".format(parameters),'\n')

grid =  GridSearchCV(Lasso(max_iter=10000), parameters, cv=5, return_train_score=True, scoring='r2', n_jobs=-1)

# perform grid search cv on training data.  The CV algorithm divides this into training and validation
ls_model = grid.fit(X_train, y_train)

print('best params ',ls_model.best_params_,'\n')
print('best estimator ',ls_model.best_estimator_,'\n')
print('best validation score ', ls_model.best_score_,'\n')
print('scoring method ', ls_model.scorer_)

print("Test set accuracy score: {:.7f}".format(ls_model.score(X_test, y_test)))

#saveModel('ls_model', ls_model)

Parameter grid:
{'alpha': [1e-09, 1e-08, 1e-07]} 

best params  {'alpha': 1e-07} 

best estimator  Lasso(alpha=1e-07, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False) 

best validation score  0.020122921851167975 

scoring method  make_scorer(r2_score)
Test set accuracy score: 0.0224324
